In [1]:
import numpy.random as rdm
import numpy as np
import matplotlib.pyplot as plt
import scipy.fftpack

In [2]:
def inicial(N):
    state = 2*rdm.randint(2,size=N)-1
    return state

In [3]:
# campo aleatorio gaussiano
def grf(N):
    alpha = 1.0
    delta = 1.0
    flag_normalize = True

    k_idx = np.mgrid[:N] - int( (N + 1)/2 )
    k_idx = scipy.fftpack.fftshift(k_idx)

    amplitude = np.power( k_idx**2 + 1e-10, -alpha/4.0 )
    amplitude[0] = 0

    noise = rdm.normal(size=(N)) \
        + 1j * rdm.normal(size=(N))

    gfield = np.fft.ifft(noise * amplitude).real

    if flag_normalize:
        gfield = gfield - np.mean(gfield)
        gfield = delta*gfield/np.std(gfield)

    return gfield

In [4]:
def grfM(N, alpha=1.0, delta=1.0, flag_normalize=True):
    
    '''
    shift the zero frequency component to the center of the spectrum
    numpy.mgrid = <numpy.lib.index_tricks.MGridClass object>
    An instance which returns a dense multi-dimensional “meshgrid”.
    '''
    k_idx = np.mgrid[:N] - (N + 1)//2
    k_idx = scipy.fftpack.fftshift(k_idx)

    amplitude = np.power( k_idx**2 + 1e-10, -alpha/4.0 )
    amplitude[0] = 0

    noise = rdm.normal(size=(N)) + 1j * rdm.normal(size=(N)) # complex noise

    gfield = np.fft.ifft(noise * amplitude).real # inverse Fourier transform

    # normalize the field to have zero mean and unit standard deviation
    if flag_normalize:
        gfield = gfield - np.mean(gfield)
        gfield = delta*gfield/np.std(gfield)

    return gfield

In [5]:
# Spin flip
def spin_flip(beta, config):
    nb = 0
    index = []
    rspin = []
    print(N, config)
    for i in range(N):
        a = rdm.randint(N)
        index.append(a)
        s = config[a]
        print(a,s, end=' ')
        nb = 0
        for j in range(N):
            if j != a:
                nb += config[(j)%N]
        print(nb, end=' ')
        cost = 2*s*nb # cost = 2*E (E1 - E0 = 2E)
        print(cost, end=' ')
        if cost < 0:
            s *= -1
        else:
            r = rdm.rand()
            rspin.append(r)
            if r < np.exp(-cost*beta):
                print("no else: {} -{} ".format(cost*beta, np.exp(-cost*beta)), end=' ')
                s *= -1
        config[a] = s
        print(s)
    return config, index, rspin

In [6]:
# Spin flip
def spin_flipM(N, rindex, rspin, beta,config):
    nb = 0
    #rindex = rdm.randint(N, size=N)
    print(N, config)
    rid = 0
    for a in rindex:
        s = config[a]
        print(a,s, end=' ')
        nb = config[:a].sum() + config[(a+1):].sum()
        print(nb, end=' ')
        cost = 2*s*nb # cost = 2*E (E1 - E0 = 2E)
        print(cost, end=' ')
        if cost < 0:
            s *= -1
        else:
            r = rspin[rid]
            rid += 1
            if r < np.exp(-cost*beta):
                print("no else: {} -{} ".format(cost*beta, np.exp(-cost*beta)), end=' ')
                s *= -1
        config[a] = s
        print(s)
    return config

In [7]:
# Energia
def energy(config,alpha):
    e = 0
    h = grf(N)
    for i in range(N):
        for j in range(i,N):
            if i != j:
                e += -config[i]*config[j]/(abs(i-j)**alpha) - h[i]*config[i]
    return e

In [8]:
# Magnetizacao
def magnetization(config):
    m = np.sum(config)
    return m

In [9]:
# Densidade de defeito
def defeito(config):
    d = 0
    h = grf(N)
    for i in range(N):
        if config[i] != h[i]:
            d += 1
    return d

In [10]:
nt     = 100        #  number of temperature points
N      = 2**4       #  size of the lattice, N
alpha = 2.          #  exponent of the long range interaction
eqSteps = 5       #  number of MC sweeps for equilibration
mcSteps = 500       #  number of MC sweeps for calculation

# valores de temperatura
T       = np.linspace(1, 200, nt);


Time    = np.linspace(0, 100, mcSteps)
E,M,C,X,Cr,Rho = np.zeros(nt), np.zeros(nt), np.zeros(nt), np.zeros(nt), np.zeros(nt), np.zeros(nt)
n1, n2  = 1.0/(mcSteps*N*N), 1.0/(mcSteps*mcSteps*N*N)

In [11]:
rdm.seed(1234567890)

In [12]:
config = inicial(N)
configIni = config.copy()
print(config)

[-1  1 -1  1 -1  1  1  1  1 -1 -1  1 -1 -1 -1 -1]


In [13]:
#primeira iteração
ttL = list(range(nt))
tt = ttL[0]

In [14]:
E1 = M1 = E2 = M2 = 0

In [15]:
iT=1/T[tt] 
iT2=iT*iT

In [16]:
for i in range(1):         # equilibrate
    config, index, rspin = spin_flip(iT,config)
    print(i,config)
print(config)

16 [-1  1 -1  1 -1  1  1  1  1 -1 -1  1 -1 -1 -1 -1]
9 -1 -1 2 no else: 2.0 -0.1353352832366127  1
15 -1 1 -2 1
12 -1 3 -6 1
0 -1 5 -10 1
15 1 5 10 1
11 1 5 10 1
6 1 5 10 1
14 -1 7 -14 1
8 1 7 14 1
1 1 7 14 1
7 1 7 14 1
10 -1 9 -18 1
13 -1 11 -22 1
15 1 11 22 1
10 1 11 22 1
2 -1 13 -26 1
0 [ 1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1]
[ 1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1]


In [17]:
rdm.seed(1234567890)
configM = inicial(N)
print(configM)
print(configIni - configM)

[-1  1 -1  1 -1  1  1  1  1 -1 -1  1 -1 -1 -1 -1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [18]:
for i in range(1):         # equilibrate
    configM = spin_flipM(N, index, rspin, iT,configM)
    print(i,configM)
print(configM)
print(config - configM)

16 [-1  1 -1  1 -1  1  1  1  1 -1 -1  1 -1 -1 -1 -1]
9 -1 -1 2 no else: 2.0 -0.1353352832366127  1
15 -1 1 -2 1
12 -1 3 -6 1
0 -1 5 -10 1
15 1 5 10 1
11 1 5 10 1
6 1 5 10 1
14 -1 7 -14 1
8 1 7 14 1
1 1 7 14 1
7 1 7 14 1
10 -1 9 -18 1
13 -1 11 -22 1
15 1 11 22 1
10 1 11 22 1
2 -1 13 -26 1
0 [ 1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1]
[ 1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


```  
     0  1  2  3  4  5  6  7  8  9  0  1  2  3  4  5
16 [-1  1 -1  1 -1  1  1  1  1 -1 -1  1 -1 -1 -1 -1]
16 [-1  1 -1  1 -1  1  1  1  1 -1 -1  1 -1 -1 -1 -1]
a  s nb c  s 
9 -1 -1 2 no else: 2.0 -0.1353352832366127  1
9 -1 -1 2 no else: 2.0 -0.1353352832366127  1

```
